In [9]:
import tensorflow as tf
import os
import numpy as np

from pinn import PINNModel, model1, optm1
from data_gen import DataGenerator, real_u1, get_data

In [10]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [11]:
session='7'
data = get_data(True, session)
EPOCHS = data['epochs']
koefs = data['big_range']
if not data["session_exists"]:
    os.makedirs(f"../models/s{session}")

In [7]:
model = PINNModel(
    model1(), optm1(), lossf="mae", initial_weights=DataGenerator.init_name(session))

# so now there are weights at this path
model.save(DataGenerator.init_name(session))
dg = DataGenerator(data['x'], data['y'], model.predict, real_u1)

In [ ]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
dataset = dg.inner_pairs(area_grid_size), dg.border_pairs(border_grid_size)

for koef in koefs:
    model = PINNModel(
        model1(), optm1(), "mae", initial_weights=DataGenerator.init_name(session))
    # model.reset()
    model.fit(koef, *dataset, EPOCHS)
    model.save(dg.name(koef, session))